<a href="https://colab.research.google.com/github/cheatham1/BackUp/blob/main/OPBG_stance_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 10.1 MB/s eta 0:00:00


In [ ]:
#import os
!pip install -q transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 4.2 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForCausalLM

model_FlanT5 = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
tokenizer_FlanT5 = AutoTokenizer.from_pretrained("google/flan-t5-small")

# HUGE !!! Need more than 200 GB
#model_BLOOMZ = AutoModelForCausalLM.from_pretrained("bigscience/bloomz", torch_dtype="auto", device_map="auto")
#tokenizer_BLOOMZ = AutoTokenizer.from_pretrained("bigscience/bloomz")

#model_opt_iml = AutoModelForCausalLM.from_pretrained("facebook/opt-iml-30b")
#tokenizer_opt_iml = AutoTokenizer.from_pretrained("facebook/opt-iml-30b")

#model_FlanT5_XXL = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xxl")
#tokenizer_FlanT5_XXL = AutoTokenizer.from_pretrained("google/flan-t5-xxl")

#model_GPT = OpenAIGPTModel.from_pretrained("openai-gpt")
#tokenizer_GPT = OpenAIGPTTokenizer.from_pretrained("openai-gpt")


In [ ]:
modelname = "model_FlanT5"
model = model_FlanT5
tokenizer = tokenizer_FlanT5

#modelname = "model_BLOOMZ"
#tokenizer = tokenizer_BLOOMZ
##model = model_BLOOMZ

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv(
    "https://raw.githubusercontent.com/cheatham1/OPBG/main/Export2023_CleanedSelection_wUser_wEnglish_wEmojTrans_sub1.tsv", sep='\t'
)


In [ ]:
data = df.copy()

result_names = ["stanceIt", "stanceEn", "stanceEnEm", "difficult", "sarcasm", "humour", "personalExp", "news","politics", "discouragingQenglish", "discouragingQitalian"]


responses = []

#for i in range(len(df)):
#range1 = int(len(df)/4)
#for i in range(range1):
for i in range(50):

    text = data["text_clean"].iloc[i]
    text_En = df["text_clean_english"].iloc[i]
    text_EnEm = df["text_clean_emojistranslated"].iloc[i]

    prompt_textIt = "Q: Is the Tweet's stance promotional, neutral, discouraging or ambiguous towards vaccines? Tweet:"+text+ ". A:"
    prompt_textEn = "Q: Is the Tweet's stance promotional, neutral, discouraging or ambiguous towards vaccines? Tweet:"+text_En+ ". A:"
    prompt_textEnEm = "Q: Is the Tweet's stance promotional, neutral, discouraging or ambiguous towards vaccines? Tweet:"+text_EnEm+ ". A:"

    prompt_textDiff = "Q: Is the Tweet's stance towards vaccines difficult to classify? Tweet:"+text+ ". A:"
    prompt_textSarc = "Q: Is the language in the Tweet sarcastic? Tweet:"+text+ ". A:"
    prompt_textHum = "Q: Is there humour in the Tweet? Tweet:"+text+ ". A:"

    prompt_textPerso = "Q: Is the Tweet referring to a personal experience? Tweet:"+text+ ". A:"
    prompt_textNews = "Q: Is the Tweet referring to a news item? Tweet:"+text+ ". A:"
    prompt_textPolitics = "Q: Is the Tweet referring to a politician or political decisions? Tweet:"+text+ ". A:"

    prompt_dis_EnIt = "Q: Does the Tweet share a discouraging stance regards vaccines ?"+text+ ". A:"
    prompt_dis_ItIt= "Q: Il Tweet condivide una posizione scoraggiante riguardo ai vaccini?" +text+ ". A:"


    prompts = [prompt_textIt, prompt_textEn, prompt_textEnEm,
               prompt_textDiff, prompt_textSarc, prompt_textHum,
               prompt_textPerso, prompt_textNews, prompt_textPolitics,
               prompt_dis_EnIt, prompt_dis_ItIt]

    i=0
    for prompt in prompts:


      inputs = tokenizer(prompt, return_tensors="pt")
      outputs = model.generate(**inputs)
      result = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

      locals()[result_names[i]] = result

      i=i+1

    row = [text, text_En, stanceIt, stanceEn, stanceEnEm, difficult, sarcasm, humour, personalExp,
           news, politics, discouragingQenglish, discouragingQitalian]
    #print(row)
    responses.append(row)



In [ ]:
dfresponses = pd.DataFrame(responses)
dfresponses.columns= ["text","text_english", 'stanceIt',
 'stanceEn',
 'stanceEnEm',
 'difficult',
 'sarcasm',
 'humour',
 'personalExp',
 'news',
 'politics',
 'discouragingQenglish', 'discouragingQitalian']
dfresponses.shape

(50, 13)

In [ ]:
# save
from google.colab import files

filename = "responses_" + modelname + ".tsv"
dfresponses.to_csv(filename, sep="\t")
files.download(filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
dfresponses.head()

,text,text_english,stanceIt,stanceEn,stanceEnEm,difficult,sarcasm,humour,personalExp,news,politics,discouragingQenglish,discouragingQitalian,diffLangText,diffEmojis,diffLangQuestion
0,ma i morti devono farlo lo stesso il vaccino?,but the dead have to do the same vaccine?,ambiguous,ambiguous,ambiguous,no,no,no,no,no,a politician,no,Tweet condividere una posizione scoraggiante r...,True,True,False
1,Vaccini covid: in Piemonte superata la quota d...,Covid vaccines: in Piedmont the quota of 9 mil...,neutral,ambiguous,ambiguous,no,no,no,no,no,political,no,i stifu a vaccini?,False,True,False
2,Sputnik:Nel Lazio vaccinazione antinfluenzale ...,Sputnik: Mandatory flu vaccination in Lazio si...,ambiguous,ambiguous,ambiguous,no,Italian,"No, it's not.",no,no,political,no,i stanno ai vaccini?Sputnik:Ne,True,True,False
3,Quasi due milioni di liguri hanno ricevuto la ...,Almost two million Ligurians have received the...,neutral,ambiguous,ambiguous,no,no,i am not a fan of vaccino,no,no,political,no,i stifu ad una posizione scoraggiante,False,True,False
4,"Vaccino Covid, il piano per gli under 65 in Li...","Covid vaccine, the plan for the under 65s in L...",ambiguous,ambiguous,ambiguous,no,no,no,no,no,political,i'm not a fan of vaccines,i sapete ai vaccini?,True,True,False


In [ ]:
# Does translating the text make a difference?
dfresponses['diffLangText'] = dfresponses['stanceIt']==dfresponses['stanceEn']

# Does translating the emojis make a difference?
dfresponses['diffEmojis'] = dfresponses['stanceEn']==dfresponses['stanceEnEm']

#Does the language the question is posed in make a difference ?
dfresponses['diffLangQuestion'] = dfresponses['discouragingQenglish']==dfresponses['discouragingQitalian']


In [ ]:
dfresponses.groupby(['stanceIt']).count()

,text,text_english,stanceEn,stanceEnEm,difficult,sarcasm,humour,personalExp,news,politics,discouragingQenglish,discouragingQitalian,diffLangText,diffEmojis,diffLangQuestion
stanceIt,,,,,,,,,,,,,,,
ambiguous,43,43,43,43,43,43,43,43,43,43,43,43,43,43,43
neutral,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7


In [ ]:
dfresponses.groupby(['stanceEn']).count()

,text,text_english,stanceIt,stanceEnEm,difficult,sarcasm,humour,personalExp,news,politics,discouragingQenglish,discouragingQitalian,diffLangText,diffEmojis,diffLangQuestion
stanceEn,,,,,,,,,,,,,,,
ambiguous,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45
neutral,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5


In [ ]:
dfresponses.groupby(['stanceEnEm']).count()

,text,text_english,stanceIt,stanceEn,difficult,sarcasm,humour,personalExp,news,politics,discouragingQenglish,discouragingQitalian,diffLangText,diffEmojis,diffLangQuestion
stanceEnEm,,,,,,,,,,,,,,,
ambiguous,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45
neutral,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5


In [ ]:
# Check if the stance label is different if the text is in Italian oe English
dfresponses.groupby(['diffLangText']).count()

,text,text_english,stanceIt,stanceEn,stanceEnEm,difficult,sarcasm,humour,personalExp,news,politics,discouragingQenglish,discouragingQitalian,diffEmojis,diffLangQuestion
diffLangText,,,,,,,,,,,,,,,
False,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8
True,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42


In [ ]:
# Check if the stance label is different if the emojis are tranlated in the text
dfresponses.groupby(['diffEmojis']).count()

,text,text_english,stanceIt,stanceEn,stanceEnEm,difficult,sarcasm,humour,personalExp,news,politics,discouragingQenglish,discouragingQitalian,diffLangText,diffLangQuestion
diffEmojis,,,,,,,,,,,,,,,
True,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50


In [ ]:
# Check if the language the question is posed in make a difference ?
dfresponses.groupby(['diffLangQuestion']).count()

,text,text_english,stanceIt,stanceEn,stanceEnEm,difficult,sarcasm,humour,personalExp,news,politics,discouragingQenglish,discouragingQitalian,diffLangText,diffEmojis
diffLangQuestion,,,,,,,,,,,,,,,
False,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50


In [ ]:
dfresponses.groupby(['news']).count()

,text,text_english,stanceIt,stanceEn,stanceEnEm,difficult,sarcasm,humour,personalExp,politics,discouragingQenglish,discouragingQitalian,diffLangText,diffEmojis,diffLangQuestion
news,,,,,,,,,,,,,,,
Continuo a leggere di gentaglia che scrive,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Is it a news item?,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
Tweet is referring to a news item.,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Tweet: “Quest'uomo è un #vaccino vivente,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
no,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33
yes,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12


In [ ]:
dfresponses.groupby(['politics']).count()

,text,text_english,stanceIt,stanceEn,stanceEnEm,difficult,sarcasm,humour,personalExp,news,discouragingQenglish,discouragingQitalian,diffLangText,diffEmojis,diffLangQuestion
politics,,,,,,,,,,,,,,,
I richiami troppo frequenti sono controproducenti,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
#HealthDay #HealthDay #HealthDay #HealthDay #HealthDay #HealthDay #,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
'La seconda dose deve essere eterologa',1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Draghi sta chiamando a sé tutta la,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
La premier neozelandese Jacinda Ardern Schwab:,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Politico,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
Presidente. (Mattarella visita scuola con alta presen,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Tweet is referring to a politician or political decisions,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
a political decision,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9


In [ ]:
# "Is the tweet discouraging ? Question posed in English"
dfresponses.groupby(['discouragingQenglish']).count()

,text,text_english,stanceIt,stanceEn,stanceEnEm,difficult,sarcasm,humour,personalExp,news,politics,discouragingQitalian,diffLangText,diffEmojis,diffLangQuestion
discouragingQenglish,,,,,,,,,,,,,,,
,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
I richiami troppo frequenti sono controproducenti,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Is it a disgrace to say that vaccines are not a threat to,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Is it a sexy stance?,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
No,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"No, I am not a doctor, but I am a doctor. I am",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"No, I am not a fan of vaccines.",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"No, I'm not a fan of vaccines.",2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
"No, it is not a 'vaccine' stance",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [ ]:
# "Is the tweet discouraging ? Question posed in Italian"

dfresponses.groupby(['discouragingQitalian']).count()

,text,text_english,stanceIt,stanceEn,stanceEnEm,difficult,sarcasm,humour,personalExp,news,politics,discouragingQenglish,diffLangText,diffEmojis,diffLangQuestion
discouragingQitalian,,,,,,,,,,,,,,,
,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
#HealthDay #HealthDay #HealthDay #HealthDay #HealthDay #HealthDay #,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
'Italia grazie al greenpass..,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
'Italiana sé a sé tutta la,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
'La seconda dose deve essere eterologa',1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
'Sono veramente schifata Continuo a legger,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"'Venezia, ULSS 3 Serenissima: #",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
In GranBretagna è uscito una documenta con le,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Is it a stifle posizione scoraggiante r,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [ ]:
dfresponses[dfresponses['diffLangQuestion'] == True]

,text,text_english,stanceIt,stanceEn,stanceEnEm,difficult,sarcasm,humour,personalExp,news,politics,discouragingQenglish,discouragingQitalian,diffLangText,diffEmojis,diffLangQuestion
0,ma i morti devono farlo lo stesso il vaccino?,but the dead have to do the same vaccine?,ambiguous,ambiguous,ambiguous,no,no,no,no,no,a politician,no,Tweet condividere una posizione scoraggiante r...,True,True,False
1,Vaccini covid: in Piemonte superata la quota d...,Covid vaccines: in Piedmont the quota of 9 mil...,neutral,ambiguous,ambiguous,no,no,no,no,no,political,no,i stifu a vaccini?,False,True,False
2,Sputnik:Nel Lazio vaccinazione antinfluenzale ...,Sputnik: Mandatory flu vaccination in Lazio si...,ambiguous,ambiguous,ambiguous,no,Italian,"No, it's not.",no,no,political,no,i stanno ai vaccini?Sputnik:Ne,True,True,False
3,Quasi due milioni di liguri hanno ricevuto la ...,Almost two million Ligurians have received the...,neutral,ambiguous,ambiguous,no,no,i am not a fan of vaccino,no,no,political,no,i stifu ad una posizione scoraggiante,False,True,False
4,"Vaccino Covid, il piano per gli under 65 in Li...","Covid vaccine, the plan for the under 65s in L...",ambiguous,ambiguous,ambiguous,no,no,no,no,no,political,i'm not a fan of vaccines,i sapete ai vaccini?,True,True,False
5,GORDON Futura taglia media 2 mesi Si affi...,GORDON Futura medium size 2 months He trusts v...,ambiguous,ambiguous,ambiguous,no,no,"io Se interessati, inviare un messaggio di pre...",no,no,political,"i'm not a fan of vaccines, but i '",i stanno ai vaccini? GORDON Futura,True,True,False
6,"#Coronaviriusitalia ma io mi chiedo, se faccia...","#Coronaviriusitalia but I wonder, if we make a...",ambiguous,ambiguous,ambiguous,Is it difficult to classify vaccines?,no,Is there humour in the Tweet?,yes,yes,political decisions,Is it a disgrace to say that vaccines are not ...,Tweet condivide una posizione scoraggiante rig...,True,True,False
7,Go un mal di testa assurdo credo per il vaccin...,I have an absurd headache I think from the vac...,ambiguous,ambiguous,ambiguous,no,no,no,no,no,a political decision,no,Is it a stifle posizione scoraggiante r,True,True,False
8,#obiang del #sassuolo sparito nel nulla dall'e...,#obiang from #sassuolo disappeared into thin a...,ambiguous,ambiguous,ambiguous,stance towards vaccines,no,omertà assoluta del giornalismo. #,no,no,a politician,no,,True,True,False
9,"Covid, più di 100 mila vaccini nelle farmacie ...","Covid, more than 100 thousand vaccines in #Sic...",ambiguous,ambiguous,ambiguous,stance towards vaccines,Italian,...........................,yes,yes,political decisions,no,Tweet condividere una posizione scoraggiante r...,True,True,False
